In [1]:
import pandas as pd
import json
import getpass
import requests
import datetime
import os


In [2]:
WB_TOKEN = os.getenv('WB_TOKEN')
headers = {'Authorization': WB_TOKEN}

In [3]:
subj_chars_url = 'https://content-api.wildberries.ru/content/v2/object/charcs/3756'
resp = requests.get(subj_chars_url, headers=headers)


In [4]:
d = json.loads( resp.text)

In [5]:
d

{'data': [{'charcID': 17596,
   'subjectName': 'Пакеты упаковочные',
   'subjectID': 3756,
   'name': 'Материал изделия',
   'required': False,
   'unitName': '',
   'maxCount': 3,
   'popular': True,
   'charcType': 1},
  {'charcID': 88952,
   'subjectName': 'Пакеты упаковочные',
   'subjectID': 3756,
   'name': 'Вес товара с упаковкой (г)',
   'required': False,
   'unitName': 'г',
   'maxCount': 0,
   'popular': False,
   'charcType': 4},
  {'charcID': 90630,
   'subjectName': 'Пакеты упаковочные',
   'subjectID': 3756,
   'name': 'Высота предмета',
   'required': False,
   'unitName': 'см',
   'maxCount': 0,
   'popular': False,
   'charcType': 4},
  {'charcID': 90652,
   'subjectName': 'Пакеты упаковочные',
   'subjectID': 3756,
   'name': 'Глубина предмета',
   'required': False,
   'unitName': 'см',
   'maxCount': 0,
   'popular': False,
   'charcType': 4},
  {'charcID': 90673,
   'subjectName': 'Пакеты упаковочные',
   'subjectID': 3756,
   'name': 'Ширина предмета',
   'requir

In [6]:
names = [n['name'] + "->" + str(n['required']) for n in d['data']]

In [11]:
import os
import re
import sqlite3

def gather_products_from_folder_and_db(db_path, images_folder):
    pattern = re.compile(r'^(\d+)-(\d+)\.jpg$', re.IGNORECASE)
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    products_list = []

    for filename in os.listdir(images_folder):
        match = pattern.match(filename)
        if not match:
            continue

        product_id, quantity_str = match.groups()
        cursor.execute("SELECT * FROM products WHERE product_id = ?", (product_id,))
        row = cursor.fetchone()

        if not row:
            continue

        (row_id, db_product_id, name, price_str, description, material,
         length_mm, width_mm, thickness_mkm, volume_l, cardboard_brand,
         cardboard_type, color, has_handles, box_shape, weight_g,
         transport_volume_m3, trade_mark, manufacturer_country) = row

        # Преобразование размеров: берем значения напрямую без дополнительных вычислений.
        height_cm = round(float(length_mm), 2) if length_mm else None
        width_cm  = round(float(width_mm), 2) if width_mm else None

        dimensions = {
            "length": int(height_cm) if height_cm else 0,
            "width": int(width_cm) if width_cm else 0,
            "height": 2
        }

        packaging_qty = volume_l if volume_l else "100"

        options = []

        options.append({
            "name": "Количество предметов в упаковке",
            "value": f"{packaging_qty} шт",
            "is_variable": True,
            "charc_type": 1,
            "variable_values": [f"{packaging_qty} шт"]
        })

        if color:
            options.append({
                "name": "Цвет",
                "value": color,
                "is_variable": True,
                "charc_type": 1,
                "variable_values": [color]
            })

        if material:
            options.append({
                "name": "Материал изделия",
                "value": material,
                "charc_type": 1
            })

        if height_cm:
            options.append({
                "name": "Высота предмета",
                "value": f"{height_cm} см",
                "charc_type": 4
            })

        if width_cm:
            options.append({
                "name": "Ширина предмета",
                "value": f"{width_cm} см",
                "charc_type": 4
            })

        options.append({
            "name": "Вид замка пакета",
            "value": "липкая лента",
            "charc_type": 1
        })

        if manufacturer_country:
            options.append({
                "name": "Страна производства",
                "value": manufacturer_country,
                "charc_type": 1
            })

        if cardboard_brand:
            options.append({
                "name": "Комплектация",
                "value": f"{cardboard_brand} - {packaging_qty} шт",
                "charc_type": 1
            })

        if weight_g:
            options.append({
                "name": "Вес товара с упаковкой",
                "value": f"{weight_g} г",
                "charc_type": 4
            })

        variant = {
            "vendorCode": f"bopp_{product_id}_{packaging_qty}",
            "title": name,
            "description": description or "Описание отсутствует",
            "options": options,
            "dimensions": dimensions
        }

        product_object = {
            "subjectID": 3756,
            "variants": [variant]
        }

        products_list.append(product_object)

    conn.close()
    return products_list


In [12]:

db_file = "bopp.db"  # путь к вашей базе данных
folder_path = "bopp_images"  # путь к папке с изображениями

result = gather_products_from_folder_and_db(db_file, folder_path)
for item in result:
    print(item)
# Здесь item — уже готовая структура для Wildberries API.
# Дальше можно вызывать свой метод create_product_cards(result), как в предыдущем примере.


{'subjectID': 3756, 'variants': [{'vendorCode': 'bopp_9604590_50', 'title': 'Пакет ПВД с замком слайдер 15*20 см, 70 мкм, матовый, 50 шт.', 'description': 'Матовый пакет-слайдер шириной 150 мм, высотой 200 мм и толщиной 70 мкм, с вентиляционными отверстиями. Изготовлен из ПВД полиэтилена. Стоимость указана за 50 штук.\nПрименяется для упаковки и хранения:\nодежды и обуви малых размеров (соответствует требованиям маркетплейсов);\nфруктов и овощей;\nбытовой химии;\nдокументов;\nмедикаментов и косметических средств;\nсредств личной гигиены.\nПреимущества:\nмногоразовый;\nпрочность;\nпростой в использовании замок;\nпрезентабельный вид пакета;\nнизкая цена.', 'options': [{'name': 'Количество предметов в упаковке', 'value': '50 шт', 'is_variable': True, 'charc_type': 1, 'variable_values': ['50 шт']}, {'name': 'Материал изделия', 'value': 'ПВД', 'charc_type': 1}, {'name': 'Высота предмета', 'value': '15.0 см', 'charc_type': 4}, {'name': 'Ширина предмета', 'value': '20.0 см', 'charc_type': 4},

In [13]:
API_URL = "https://content-api.wildberries.ru/content/v2/cards/upload"



def create_product_cards(products):
  
    try:
        response = requests.post(API_URL, json=products, headers=headers)
        response_data = response.json()
        
        if response.status_code == 200 and not response_data.get("error", True):
            print("Карточки товаров успешно созданы.")
        else:
            print(f"Ошибка: {response_data.get('errorText', 'Неизвестная ошибка')}")
            print("Дополнительные ошибки:", response_data.get("additionalErrors", {}))
        
        return response_data
    except requests.RequestException as e:
        print(f"Ошибка при выполнении запроса: {e}")
        return None

In [14]:
resp = create_product_cards(result)

Карточки товаров успешно созданы.
